##### In this example, we will write the output of the Sycamore from pdf to `FAISS` vector database using Langchain.


##### The Aryn Partitioner in this job is configured to use the Aryn Partitioning Service to provide fast, GPU-powered performance. Go to [aryn.ai/sign-up ](aryn.ai/sign-up) to get a free API key for the service. This is the recommended configuration.

##### You can also run the Aryn Partitioner locally by setting `use_partitioning_service` to `False`. Though you can use CPU to run the Aryn Partitioner, it is recommended to use an NVIDIA GPU for good performance.



In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms.openai import OpenAI
from langchain.callbacks import get_openai_callback

from pathlib import Path
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import sycamore 
from sycamore.data import Document
from sycamore.transforms.partition import ArynPartitioner


In [ ]:
# requirements to be added 
#  faiss-cpu==1.7.4
#  langchain-community


In [ ]:
from sycamore.utils.aryn_config import ArynConfig, DEFAULT_PATH
assert ArynConfig.get_aryn_api_key() != "", f"Unable to find aryn API key.  Looked in {DEFAULT_PATH}"

if the above assertion fails, you can either set the environment variable ARYN_API_KEY and restart jupyter
or make a yaml file at the specified path in the assertion error that looks like:

```
aryn_token: "YOUR-ARYN-API-KEY"
```

It is unsafe, but if neither of those options work, you can put it in this notebook with
```
import os
os.environ["ARYN_API_KEY"] = "UNSAFE-ARYN-API-KEY-LOCATION" 
```

but beware that it is easy to accidentally commit the notebook file and have it include your key.

In [ ]:
if os.path.exists("/.dockerenv"):
    # Running in Docker.
    work_dir = "/app/work/docker_volume"
else:
    # Running outside of docker. This will land under notebooks/data/
    work_dir = "./data"
    

In [ ]:

os.makedirs(work_dir, exist_ok = True)
metadata = {}
for f in ["2306.07303"]:
    path = os.path.join(work_dir, f + ".pdf")
    url = os.path.join("http://arxiv.org/pdf", f)
    if not Path(path).is_file():
        print("Downloading {} to {}".format(url, path))
        subprocess.run(["curl", "-o", path, url])


In [ ]:
context = sycamore.init()
work_dirs = ['./data/2306.07303.pdf']
pdf_docset = context.read.binary(work_dirs, binary_format="pdf")


partitioned_docset = pdf_docset.partition(
    partitioner=ArynPartitioner(threshold=0.35, extract_table_structure=True) 
        )


(pid=31845) INFO:root:Spurious log 1: Verifying that log messages are propogated
(raylet) [2024-08-01 00:42:47,160 E 13985 13985] (raylet) node_manager.cc:2963: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: b8427498e2278db9e5acca81e16481a65fdd58b4e4c17ff05281581f, IP: 10.0.20.45) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.0.20.45`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(pid=709) INFO:root:Spurious log 1: Veri

(raylet) The autoscaler failed with the following error:
Terminated with signal 15
  File "/home/ec2-user/.cache/pypoetry/virtualenvs/sycamore-monorepo--FwAn7mM-py3.10/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 709, in <module>
    monitor.run()
  File "/home/ec2-user/.cache/pypoetry/virtualenvs/sycamore-monorepo--FwAn7mM-py3.10/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 584, in run
    self._run()
  File "/home/ec2-user/.cache/pypoetry/virtualenvs/sycamore-monorepo--FwAn7mM-py3.10/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 438, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



In [ ]:
text = ""
for doc in partitioned_docset.take_all():
    for doci in doc.elements:
        if doci.type == "table":
            text +=  doci['table'].to_csv()
        elif doci.text_representation:
            text +=  doci.text_representation


In [ ]:

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_overlap = 200,
    chunk_size = 1000,
    length_function = len 
)

chunks = text_splitter.split_text(text)

embedding = OpenAIEmbeddings()
faiss_index = FAISS.from_texts(chunks, embedding)


In [ ]:
def questionAnswering(user_question):
    docs = faiss_index.similarity_search(user_question, k=5)
    llm = OpenAI()
    chain = load_qa_chain(llm, chain_type= "stuff")
    with get_openai_callback() as cb:
        response = chain.run(input_documents=docs, question=user_question)
        print(cb)
        print(response)


In [ ]:
questionAnswering("What is transformer?")